![](https://www.zorgkaartnederland.nl/bundles/zkncontent/images/elements/logo_zkn.svg)

In [1]:
import zorgkaart

In [2]:
import folium
import os
import pandas as pd

## Delete data folder

In [3]:
try:
    for file in os.listdir('data'):
        os.remove(os.path.join('data', file))
    os.rmdir('data')
except:
    pass

## Organisatietype
Met de functie `zorgkaart.get_types()` krijg je een overzicht van alle organisatietypen waarvoor je met de module `zorgkaart` data kan ophalen van de websit [www.zorkaartnederland.nl](www.zorkaartnederland.nl).  

De variable `aantal` geeft het aantal zorginstellingen aan in betreffende categorie. De variabele `organisatietype_url` heeft aan welke waarde je als argument dient mee te geven voor het verkrijgen van instellingsgegevens.

In [4]:
organisatietypen = zorgkaart.get_types()
organisatietypen.head()

,organisatietype,organisatietype_url,aantal
0,Abortuskliniek,abortuskliniek,14
1,Apotheek,apotheek,1900
2,Arbo- en reïntegratiebedrijf,arbo-en-reintegratiebedrijf,539
3,Artsenpraktijk voor alternatieve geneeskunde,artsenpraktijk-voor-alternatieve-geneeskunde,421
4,Astmacentrum,astmacentrum,8


## Info

In [5]:
info = zorgkaart.get_info('astmacentrum')
info

100%|██████████| 1/1 [00:00<00:00,  3.75it/s]


,zorginstelling,plaats,beoordeling,waarderingen,categorie,latitude,longitude,zknl_url
id,,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Clavadel,9.2,105,Astmacentrum,0.000000,0.000000,https://www.zorgkaartnederland.nl/zorginstelli...
3058759,Center of Excellence,Amersfoort,9.0,25,Astmacentrum,52.165986,5.407762,https://www.zorgkaartnederland.nl/zorginstelli...
109628,"Merem, locatie Hilversum",Hilversum,8.8,15,Astmacentrum,52.212541,5.201926,https://www.zorgkaartnederland.nl/zorginstelli...
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Horn,8.2,10,Astmacentrum,51.219635,5.918302,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack - Regionaal Astma en Allergiecentrum voo...,Groesbeek,NaN,0,Organisatie,51.801576,5.894875,https://www.zorgkaartnederland.nl/zorginstelli...
3051521,Raack - Regionaal Astma en Allergiecentrum voo...,Nijmegen,NaN,0,Astmacentrum,51.817159,5.831195,https://www.zorgkaartnederland.nl/zorginstelli...
3051520,Raack - Regionaal Astma en Allergiecentrum voo...,Nijmegen,NaN,0,Astmacentrum,51.823384,5.860092,https://www.zorgkaartnederland.nl/zorginstelli...
3048242,Raack - Regionaal Astma en Allergiecentrum voo...,Groesbeek,NaN,0,Astmacentrum,51.801910,5.898534,https://www.zorgkaartnederland.nl/zorginstelli...


In [6]:
os.listdir('data')

['astmacentrum_info.csv']

## Details

In [7]:
details = zorgkaart.get_details('astmacentrum')
details

100%|██████████| 8/8 [00:02<00:00,  3.55it/s]


,zorginstelling,adres,postcode,plaats,telefoon,website,wachttijden_url
id,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Klinikstrasse 6,7272,Davos Clavadel,"0041-814178000, 06-31002905 (secr. Nederland)",https://nadavos.nl/,NaN
3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,http://www.coe-amersfoort.nl,NaN
109628,"Merem, locatie Hilversum",Soestdijkerstraatweg 129,1213 VX,Hilversum,035-6881411,http://www.merem.nl,NaN
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Hornerheide 1,6085 NM,Horn,0475-587587,http://www.ciro-horn.nl,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,http://www.raack.org,NaN
3051521,Raack,Weg door Jonkerbos 100,6532 SZ,Nijmegen,024-3658708,http://www.raack.org,NaN
3051520,Raack,Geert Grooteplein Zuid 10,6525 GA,Nijmegen,024-3614415,c,NaN
3048242,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,http://www.raack.org,NaN


In [8]:
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum_info.csv']

## Samenvoegen

In [9]:
samen = zorgkaart.merge_datasets('astmacentrum', delete=False)
samen

,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,categorie,latitude,longitude,website,zorgkaart_url,wachttijden_url
id,,,,,,,,,,,,,
3035478,Nederlands Astmacentrum Davos,Davos Klinikstrasse 6,7272,Davos Clavadel,"0041-814178000, 06-31002905 (secr. Nederland)",9.2,105,Astmacentrum,0.000000,0.000000,https://nadavos.nl/,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3058759,Center of Excellence,Wiekslag 90,3815 GS,Amersfoort,033-3034590,9.0,25,Astmacentrum,52.165986,5.407762,http://www.coe-amersfoort.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
109628,"Merem, locatie Hilversum",Soestdijkerstraatweg 129,1213 VX,Hilversum,035-6881411,8.8,15,Astmacentrum,52.212541,5.201926,http://www.merem.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
9848,"CIRO, Expertisecentrum voor Chronisch Orgaanfalen",Hornerheide 1,6085 NM,Horn,0475-587587,8.2,10,Astmacentrum,51.219635,5.918302,http://www.ciro-horn.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...
10001580,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,NaN,0,Organisatie,51.801576,5.894875,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3051521,Raack,Weg door Jonkerbos 100,6532 SZ,Nijmegen,024-3658708,NaN,0,Astmacentrum,51.817159,5.831195,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3051520,Raack,Geert Grooteplein Zuid 10,6525 GA,Nijmegen,024-3614415,NaN,0,Astmacentrum,51.823384,5.860092,c,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
3048242,Raack,Nijmeegsebaan 31,6561 KE,Groesbeek,024-6859250,NaN,0,Astmacentrum,51.801910,5.898534,http://www.raack.org,https://www.zorgkaartnederland.nl/zorginstelli...,NaN


In [10]:
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum.csv', 'astmacentrum_info.csv']

In [11]:
os.remove(os.path.join('data', 'astmacentrum.csv'))
os.listdir('data')

['astmacentrum_details.csv', 'astmacentrum_info.csv']

In [12]:
samen = zorgkaart.merge_datasets('astmacentrum', delete=True)
os.listdir('data')

['astmacentrum.csv']

## Meerdere organisatietypen

In [13]:
# delete files
for file in os.listdir('data'):
    os.remove(os.path.join('data', file))

In [14]:
lijst = organisatietypen[organisatietypen.aantal <= 10]
lijst

,organisatietype,organisatietype_url,aantal
4,Astmacentrum,astmacentrum,8
8,Bevolkingsonderzoek kanker,bevolkingsonderzoek-kanker,5
9,Bloedbank,bloedbank,6
10,Brandwondencentrum,brandwondencentrum,3
37,IVF-kliniek,ivf-kliniek,10
45,Ministerie,ministerie,1
51,Orthoptistenpraktijk,orthoptistenpraktijk,4


In [15]:
for organisatietype in lijst.organisatietype_url:
    zorgkaart.get_info(organisatietype)
    zorgkaart.get_details(organisatietype)
    zorgkaart.merge_datasets(organisatietype, delete=True)

100%|██████████| 4/4 [00:01<00:00,  3.29it/s]


In [16]:
os.listdir('data')

['orthoptistenpraktijk.csv',
 'ivf-kliniek.csv',
 'bloedbank.csv',
 'brandwondencentrum.csv',
 'bevolkingsonderzoek-kanker.csv',
 'astmacentrum.csv',
 'ministerie.csv']

## Command line interface

In [17]:
# delete files
for file in os.listdir('data'):
    os.remove(os.path.join('data', file))

In [18]:
!python zorgkaart.py ziekenhuis

100%|█████████████████████████████████████████| 308/308 [01:51<00:00,  3.37it/s]


In [19]:
os.listdir('data')

['ziekenhuis_details.csv', 'ziekenhuis_info.csv', 'ziekenhuis.csv']

## Zorgkaart

In [22]:
df = pd.read_csv(os.path.join('data', 'ziekenhuis.csv'), index_col='id')
df.sample(n=5)

,zorginstelling,adres,postcode,plaats,telefoon,beoordeling,waarderingen,categorie,latitude,longitude,website,zorgkaart_url,wachttijden_url
id,,,,,,,,,,,,,
10001372,Spijkenisse Medisch Centrum,Ruwaard van Puttenweg 500,3201 GZ,Spijkenisse,0181-658888,8.7,321,Organisatie,51.846791,4.336472,http://www.spijkenissemc.nl,https://www.zorgkaartnederland.nl/zorginstelli...,NaN
111118,"Albert Schweitzer ziekenhuis, locatie Sliedrecht",Wilhelminastraat 75,3361 XV,Sliedrecht,078-6541111,8.7,15,Ziekenhuis,51.822181,4.768430,http://www.asz.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...
111942,"VieCuri, locatie Venray",Merseloseweg 130,5801 CE,Venray,0478-522222,8.5,118,Ziekenhuis,51.527334,5.960389,http://www.viecuri.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...
3029478,"Dijklander ziekenhuis, buitenpolikliniek Heerh...",Middenwaard 45,1703 SC,Heerhugowaard,072 - 5753030,NaN,0,Ziekenhuis,52.661781,4.828906,https://www.dijklander.nl/,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...
3044183,"Ziekenhuis St Jansdal, Polikliniek Nijkerk",Henri Nouwenstraat 18,3863 HV,Nijkerk,0341-463911,NaN,0,Ziekenhuis,52.210255,5.466580,https://www.stjansdal.nl,https://www.zorgkaartnederland.nl/zorginstelli...,https://www.zorgkaartnederland.nl/zorginstelli...


In [23]:
# Create map
m = folium.Map(location = [52.0912374, 5.14905414], zoom_start = 8)

# Add marker for each gym
for row in df.iterrows():
    row_values = row[1]
    location = (row_values.latitude, row_values.longitude)
    inst = row_values.zorginstelling
    adr1 = row_values.adres
    adr2 = row_values.postcode + '  ' + row_values.plaats
    tel = row_values.telefoon
    bw = '{} ({} waarderingen)'.format(row_values.beoordeling, row_values.waarderingen)
    web = '<a href={0} target="_blank"</a>{0}</a>'.format(row_values.website)
    popup = '<b>{}</b><br><i>{}</i><br><i>{}</i><br>{}<br>{}<br>{}'.format(inst, adr1, adr2, tel, web, bw)
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(m)

# Display map
display(m)